# Generative Adversarial Network (GAN)

In [ ]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchsummary import summary
torch.manual_seed(42)
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torchvision import datasets, transforms as T
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 128
noise_dim = 64
lr = 0.0002
beta_1=0.5
beta_2=0.99
epochs=50

# Load MNIST Dataset

In [ ]:
train_augs=T.Compose([T.RandomRotation((-20,+20)),
                      T.ToTensor()])
trainset=datasets.MNIST('MNIST/',download=True,train=True,transform=train_augs)
print("Total Images in Trainset is ",len(trainset))
trainloader=DataLoader(trainset,batch_size=batch_size,shuffle=True)
print("Total Batches",len(trainloader))
dataiter= iter(trainloader)
images,_=next(dataiter)
print(images.shape)
# 'show_tensor_images' : function is used to plot some of images from the batch

def show_tensor_images(tensor_img, num_images = 16, size=(1, 28, 28)):
    unflat_img = tensor_img.detach().cpu()
    img_grid = make_grid(unflat_img[:num_images], nrow=4)
    plt.imshow(img_grid.permute(1, 2, 0).squeeze())
    plt.show()
show_tensor_images(images, num_images=16)

# Discriminator Network

In [ ]:
def get_disc_block(in_channels,out_channels,kernel_size,stride):
  return nn.Sequential(
      nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=stride),
      nn.BatchNorm2d(out_channels),
      nn.LeakyReLU(0.2)
  )
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.block1=get_disc_block(1,16,(3,3),2)
    self.block2=get_disc_block(16,32,(5,5),2)
    self.block3=get_disc_block(32,64,(5,5),2)
    self.flatten=nn.Flatten()
    self.linear=nn.Linear(64,1)
  def forward(self,images):
    x1=self.block1(images)
    x2=self.block2(x1)
    x3=self.block3(x2)
    x4=self.flatten(x3)
    x5=self.linear(x4)
    return x5
D=Discriminator().to(device)
summary(D, input_size=(1,28,28))

# Generator Network

In [ ]:
def get_gen_block(in_channels,out_channels,kernel_size,stride,final_block=False):
  if final_block:
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=stride),
        nn.Tanh()
    )
  return nn.Sequential(
      nn.ConvTranspose2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=stride),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
  )

In [ ]:
class Generator(nn.Module):
  def __init__(self,noise_dim):
    super(Generator,self).__init__()
    self.noise_dim=noise_dim
    self.block_1=get_gen_block(noise_dim,256,(3,3),2)
    self.block_2=get_gen_block(256,128,(4,4),1)
    self.block_3=get_gen_block(128,64,(3,3),2)
    self.block_4=get_gen_block(64,1,(4,4),2,final_block=True)
  def forward(self,noise):
    x=noise.view(-1,self.noise_dim,1,1)
    x1=self.block_1(x)
    x2=self.block_2(x1)
    x3=self.block_3(x2)
    x4=self.block_4(x3)
    return x4

In [ ]:
G=Generator(noise_dim).to(device)
summary(G, input_size=(1,noise_dim))

In [ ]:
# Replace Random initialized weights to Normal weights

def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight, 0.0, 0.02)
        nn.init.constant_(m.bias, 0)

In [ ]:
D=D.apply(weights_init)
G=G.apply(weights_init)

# Loss Function and Load Optimizer

In [ ]:
def real_loss(disc_pred):
  criterion=nn.BCEWithLogitsLoss()
  ground_truth=torch.ones_like(disc_pred)
  loss=criterion(disc_pred,ground_truth)
  return loss
def fake_loss(disc_pred):
  criterion=nn.BCEWithLogitsLoss()
  ground_truth=torch.zeros_like(disc_pred)
  loss=criterion(disc_pred,ground_truth)
  return loss
D_opt=torch.optim.Adam(D.parameters(),lr=lr,betas=(beta_1,beta_2))
G_opt=torch.optim.Adam(G.parameters(),lr=lr,betas=(beta_1,beta_2))

# Training Loop

In [ ]:
for i in tqdm(range(epochs)):
  total_d_loss=0
  total_g_loss=0
  for real_img,_ in trainloader:
    real_img=real_img.to(device)
    noise=torch.randn(batch_size,noise_dim,device=device)
    #Discriminator
    D_opt.zero_grad()
    fake_img=G(noise)
    D_pred=D(fake_img)
    D_fake_loss=fake_loss(D_pred)
    D_pred=D(real_img)
    D_real_loss=real_loss(D_pred)
    D_loss=(D_fake_loss+D_real_loss)/2
    total_d_loss+=D_loss.item()
    D_loss.backward()
    D_opt.step()
    #Generator
    G_opt.zero_grad()
    noise=torch.randn(batch_size,noise_dim,device=device)
    fake_img=G(noise)
    D_pred=D(fake_img)
    G_loss=real_loss(D_pred)
    total_g_loss+=G_loss.item()
    G_loss.backward()
    G_opt.step()
  avg_d_loss=total_d_loss/len(trainloader)
  avg_g_loss=total_g_loss/len(trainloader)
  print("Epoch : ",i+1,"     Discriminator Loss : ",avg_d_loss,"     Generator Loss : ",avg_g_loss)
  if i is epochs-1:
    show_tensor_images(fake_img)
